In [ ]:
import os
from google.colab import drive
from os.path import join as jn
import shutil

# 구글 드라이브 접근
ROOT = "/content/drive"
try:
  drive.mount(ROOT, force_remount=True)
except:
  drive.mount(ROOT)

# 본인 경로에 맞게 수정하면 됨.
PATH = jn(ROOT, "MyDrive/Task/plastic-segmentation/Data")
# 샘플 데이터 사용 시에도 동일하게 train, test 이전까지의 경로만 수정해주면 됩니다.

# 구글 드라이브 경로에서 '/content/sample_data/data"로 복사 시
shutil.copytree(PATH +"/train", "./sample_data/data/train")
shutil.copytree(PATH +"/test", "./sample_data/data/test")

# init

In [7]:
import os
import json
from tqdm import tqdm

import numpy as np
# import pandas as pd
import cv2 as cv
from PIL import Image

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pycocotools.coco import COCO

import torch
import torch.optim as optim
from torch  import nn, Tensor

BASE = os.getcwd()
PATH = f"{BASE}/sample_data/data"   # colab
# PATH = f"{BASE}/assets/data"        # github

In [21]:
# version 1 : @qkrwjdduf159

## train
trn_path = f"{PATH}/train/annotation"
file_list = os.listdir(trn_path)

train_files = []
for file in tqdm(file_list):
    dir = trn_path + '/' + file
    json_list = glob(dir + '/' + '*.json')
    train_files.append(json_list)

train_json_list = []
for files in tqdm(train_files):
    for json_file in tqdm(files):
        train_json_list.append(json_file)

## test
tst_PATH = f"{PATH}/test/annotations"
file_list = os.listdir(tst_PATH)

test_files = []
for file in tqdm(file_list):
    dir = tst_PATH + '/' + file
    json_list = glob(dir + '/' + '*.json')
    test_files.append(json_list)

test_json_list = []
for files in tqdm(test_files):
    for json_file in tqdm(files):
        test_json_list.append(json_file)

100%|██████████| 1000/1000 [00:00<00:00, 1236163.87it/s]

100%|██████████| 1000/1000 [00:00<00:00, 1814930.33it/s]

100%|██████████| 1000/1000 [00:00<00:00, 1903041.74it/s]

100%|██████████| 100/100 [00:00<00:00, 835518.73it/s]

100%|██████████| 100/100 [00:00<00:00, 761216.70it/s]

100%|██████████| 100/100 [00:00<00:00, 903944.83it/s]

100%|██████████| 4/4 [00:00<00:00, 159.80it/s]


In [42]:
# version 2 @AshbeeKim
trn = os.path.join(PATH, "train")
tst = os.path.join(PATH, "test")

baseDF = {"kind" : [], "label" : [], "metainfo_id" : [], "feature" : [], "image_path" : [], "annot_path": []}
trn_num, tst_num = 0, 0
for fpath in tqdm([trn, tst]):
    kind = os.path.basename(fpath)
    BDIR = sorted(os.listdir(fpath))    # image, annotation(s)
    for bdir in BDIR:
        for dirs in sorted(os.listdir(jn(fpath, bdir))):
            dpath = jn(fpath, bdir, dirs)
            if bdir.lower()=='image':
                paths = sorted(glob(dpath + "/*.jpg"))
                baseDF['image_path'].extend(paths)
                kinds = [kind for cnt in range(len(paths))]
                baseDF['kind'].extend(kinds)
                if kind=='train' : trn_num += len(paths)
                elif kind=='test' : tst_num += len(paths)
                fnames = [os.path.basename(fname) for fname in paths]
                labels = list(map(lambda x: (x.split('_')[0]), fnames))
                baseDF['label'].extend(labels)
                metaIds = list(map(lambda x: int(x.split('_')[1]), fnames))
                baseDF['metainfo_id'].extend(metaIds)
                feats = list(map(lambda x: int(x.split('_')[-1][:-4]), fnames))
                baseDF['feature'].extend(feats)
            else:
                paths = sorted(glob(dpath + "/*.json"))
                baseDF['annot_path'].extend(paths)

100%|██████████| 2/2 [00:00<00:00, 55.92it/s]


*** 현재 수정 중 ***

In [43]:
baseDF['kind'][3995:trn_num]

['train', 'train', 'train', 'train', 'train']

In [45]:
baseDF['kind'][trn_num:trn_num+5]

['test', 'test', 'test', 'test', 'test']

In [46]:
# cotributed by @qkrwjdduf156, @AshbeeKim
import os
import numpy as np
import torch
from PIL import Image

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, base_info, transforms = None, mode = 'train', train_size = None, test_size = None):
        self.mode = mode
        self.infos = base_info
        self.transforms = transforms
        self.train_size = train_size
        self.test_size = test_size
        
    def __getitem__(self, idx):
        if self.mode == 'train':
            json_path = (self.infos['annot_path'])[:self.train_size][idx]

            with open(json_path, 'r') as f:
                annot = json.loads(f.read())

            image_path = (self.infos['image_path'])[:self.train_size][idx]
            image = Image.open(image_path).convert('RGB')

            boxes = []
            segmentations = []
            labels = []
            for i in range(len(annot['annotations'])):

                segmentation = annot['annotations'][i]['segmentation'][0]
                bbox = annot['annotations'][i]['bbox']
                label = torch.tensor(annot['annotations'][i]['category_id'], dtype = np.unit8)
                xmin, ymin, width, height = bbox[0],bbox[1],bbox[2],bbox[3]
                xmin, ymin, xmax, ymax = xmin, ymin, xmin + width, ymax + height
                
                boxes.append([xmin, ymin, xmax, ymax])
                segmentations.append([segmentation])
                labels.append(label)

            target = {}
            target['boxes'] = boxes
            target['labels'] = labels
            target['segmentation'] = segmentations
            target['image_id'] = image_path.split('/')[-1]

            if self.transforms is not None:
                img, target = self.transforms(img, target)

            return img, target

        elif self.model == 'test':
            st_num = self.train_size
            ed_num = self.train_size + self.test_size

            json_path = ((self.infos['annot_path'])[st_num:ed_num])[idx]
            with open(json_path, 'r') as f:
                annot = json.loads(f.read())

            image_path = ((self.infos['image_path'])[st_num:ed_num])[idx]
            image = Image.open(image_path).convert('RGB')
            
            target = {}
            target['image_id'] = image_path.split('/')[-1]
            return image, target
        
    def __len__(self):
        if self.mode == 'train':
            return self.train_size
        elif self.mode == 'test':
            return self.test_size

In [47]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

train_dataset = CustomDataset(baseDF, mode = 'train', train_size=trn_num, test_size=tst_num)
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)

num_classes = 4
in_features = model.roi_heads.box_predictor.cls_score.in_features

model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]